In [1]:
import tensorflow as tf
from enet import ENet, ENet_arg_scope
import numpy as np
slim = tf.contrib.slim

checkpoint_dir = "./checkpoint"
checkpoint = tf.train.latest_checkpoint(checkpoint_dir)

with tf.Graph().as_default() as graph:
    image = tf.placeholder(dtype=tf.float32, shape=(1, 360, 480, 3), name="image_placeholder")
    
    with slim.arg_scope(ENet_arg_scope()):
        logits, probabilities = ENet(image,
                                     num_classes=12,
                                     batch_size=1,
                                     is_training=False, # was True
                                     reuse=None,
                                     num_initial_blocks=1,
                                     stage_two_repeat=2,
                                     skip_connections=False)
    
    variables_to_restore = slim.get_variables_to_restore()
    saver = tf.train.Saver(variables_to_restore)

    #sv = tf.train.Supervisor(logdir=None, init_fn=restore_fn)
    
    #with sv.managed_session() as sess:
    with tf.Session(graph=graph) as sess:
        #saver.restore(sess, checkpoint)
        for var_name, _ in tf.contrib.framework.list_variables(checkpoint_dir):
            var = tf.contrib.framework.load_variable(checkpoint_dir, var_name)
            
            #Rename loaded variables
            new_name = var_name
            if new_name.startswith('ENet/'):
                new_name = new_name[5:]
                var = tf.Variable(var, name=new_name)
                #print("Renamed " + var_name + " to " + new_name)
            #else:
            #    print("Variable " + var_name + " not renamed")
                
        sess.run(tf.global_variables_initializer())

        predictions = tf.argmax(probabilities, -1)
        predictions = tf.cast(predictions, tf.float32)
        
        # See https://blog.metaflow.fr/tensorflow-how-to-freeze-a-model-and-serve-it-with-a-python-api-d4f3596b3adc
        frozen_graph_def = tf.graph_util.convert_variables_to_constants(
             sess
            ,sess.graph_def
            ,[predictions.name.split(":")[0]] #list of outputs
        )
        
    with tf.Session(graph=tf.Graph()) as sess:
        # Cheeky? >_>
        tf.import_graph_def(
             frozen_graph_def
            ,return_elements=[predictions.name]
            ,name=''
        )
        
        #for op in sess.graph.get_operations():
        #    print(op.name)
        
        image = sess.graph.get_tensor_by_name("image_placeholder:0")
        output = sess.graph.get_tensor_by_name(predictions.name)
        segmentations = sess.run(output, feed_dict={image: np.random.rand(1, 360, 480, 3)})
        
        print("Writing tensorboard log")
        tf.summary.FileWriter('./log', graph=sess.graph).flush()

        print("Writing SavedModel")
        builder = tf.saved_model.builder.SavedModelBuilder('./logSM')
        builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING])
        builder.save()

        
    print("Done")

INFO:tensorflow:Froze 537 variables.
Converted 537 variables to const ops.
Writing tensorboard log
Writing SavedModel
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./logSM/saved_model.pb
Done
